# Observational spatial analysis


In [ ]:
%load_ext autoreload
%autoreload 2

import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LatitudeFormatter, LongitudeFormatter
import geopandas as gp
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import os
import xarray as xr

from unseen import fileio, independence, similarity, time_utils

import cfg, spatial_plots

In [ ]:
# Optional parameters
# (This cell is tagged "parameters")
dpi = 300
shapefile = None
shape_overlap = 0.1
alpha = 0.05
time_dim = "time"
lat_dim = "lat"
lon_dim = "lon"
similarity_test = "ks"
gev_relative_fit_test = "bic"
time_agg = "maximum"
covariate_year = 2024
start_year = 1961
base_period = [1961, 2020]  # for covariate

In [ ]:
# Required parameters
kwargs = locals()
assert "metric" in kwargs, "Must provide a metric name"
assert "var" in kwargs, "Must provide a variable name"
assert "model_name" in kwargs, "Must provide a model name"

assert os.path.isfile(
    obs_file
), f"Must provide an observations data file (papermill option -p obs_file [filepath])"
assert os.path.isfile(
    file_list
), "Must provide the input model files list (papermill option -p file_list [filepath])"
assert os.path.isfile(
    model_file
), "Must provide an model data file (papermill option -p model_file [filepath])"
assert os.path.isfile(
    independence_file
), "Must provide an independence min lead file (papermill option -p independence_file [filepath])"
if plot_additive_bc:
    assert os.path.isfile(
        model_add_bc_file
    ), "Must provide a model additive bias corrected data file (papermill option -p model_add_bc_file [filepath])"
if plot_multiplicative_bc:
    assert os.path.isfile(
        model_mulc_bc_file
    ), "Must provide a model multiplicative bias corrected data file (papermill option -p model_mulc_bc_file [filepath])"
assert os.path.isfile(
    similarity_raw_file
), "Must provide an raw data similarity test file (papermill option -p similarity_raw_file [filepath])"
assert os.path.isfile(
    similarity_add_bc_file
), "Must provide an additive bias corrected similarity test file (papermill option -p similarity_add_bias_file [filepath])"
assert os.path.isfile(
    similarity_mulc_bc_file
), "Must provide an multiplicative bias corrected similarity test file (papermill option -p similarity_mulc_bias_file [filepath])"

## Observational data

In [ ]:
obs_ds = fileio.open_dataset(obs_file)
event_times = np.vectorize(time_utils.str_to_cftime)(
    model_ds.event_time, model_ds.time.dt.calendar
)
model_ds["event_time"] = (model_ds.event_time.dims, event_times)
obs_ds

In [ ]:
# Select observations within the model initialisation times # todo: check this
model_start_year = model_ds.time.isel({lead_dim: 0, init_dim: 0}).dt.year.load().item()
obs_ds = obs_ds.where(obs_ds.time.dt.year >= model_start_year, drop=True)
obs_ds = obs_ds.dropna("time", how="all")
obs_ds

In [ ]:
obs_max_event = obs_ds[var].max().load().item()
obs_max_event_loc = (
    obs_ds[var].where(obs_ds[var].load() == obs_max_event, drop=True).squeeze()
)
obs_max_event_loc.load()

## Spatial Maps

In [ ]:
# Store plot related variables using the InfoSet class

info = cfg.InfoSet(
    name=model_name,
    metric=metric,
    fig_dir=fig_dir,
    file=model_file,
    ds=model_ds,
    ds_obs=obs_ds,
    bias_correction=None,
)

In [ ]:
# Load GEV parameters
covariate = obs_ds[time_dim].dt.year
# base_period = [model_ds.time.dt.year.min().load().item(), obs_ds.time.dt.year.max().load().item()]
times = xr.DataArray(base_period, dims=info.time_dim)

In [ ]:
dparams_s = xr.open_dataset(gev_params_stationary_file)[var]
dparams_ns = xr.open_dataset(gev_params_nonstationary_file)[var]
# Ensure model ds and GEV parameters have the same lat/lon grid (hard to fit a
# GEV at ocean points - usually apply a mask that drops some lat/lons)
obs_ds = model_add_bc_ds_stacked.sel(lat=dparams_ns.lat, lon=dparams_ns.lon)
print(dparams_s)
print(dparams_ns)

# Year when max/min event occured

In [ ]:
spatial_plots.plot_map_event_year(info, obs_ds, time_agg)

## Most common month for max/min event

In [ ]:
spatial_plots.plot_map_event_month_mode(info, obs_ds)


## Map of metric median

In [ ]:
spatial_plots.plot_map_time_agg(info, obs_ds, "median")

# Map of metric maximum or minimum

In [ ]:
spatial_plots.plot_map_time_agg(info, obs_ds, time_agg)

## Annual reccurence of observed max/min event

### Empirical return period

In [ ]:
spatial_plots.plot_map_obs_ari_empirical(
    info,
    obs_ds,
    None,
    covariate=covariate_year,
    time_agg=time_agg,
)

### GEV-based return period

In [ ]:
spatial_plots.plot_map_obs_ari(
    info,
    obs_ds,
    None,
    dparams_ns,
    covariate=covariate_year,
    time_agg=time_agg,
)

## GEV parameter trends

In [ ]:
for param in ["scale", "location"]:
    spatial_plots.plot_map_gev_param_trend(info, obs_ds, dparams_ns, param=param)

## Annual exceedance probability 
### Empirical exceedance probability

In [ ]:
# ARI: 10, 100, 1000 years (i.e., 10% AEP is equiv to a 1 in 10 year event)
aep_list = [10, 1, 0.1]
for aep in aep_list:
    spatial_plots.plot_map_aep_empirical(
        info,
        obs_ds,
        aep=aep,
    )

### GEV-based exceedance probability

In [ ]:
for aep in aep_list:
    spatial_plots.plot_map_aep(
        info,
        obs_ds,
        dparams_ns,
        times,
        aep=aep,
    )

## Probability of breaking the observed record

In [ ]:
spatial_plots.plot_map_new_record_probability(
    info,
    obs_ds,
    obs_ds,
    dparams_ns,
    covariate_year,
    time_agg,
    ari=10,
)